In [1]:
import asyncio
import aiomysql

loop = asyncio.get_event_loop()

In [2]:
DATABASE_USER = 'root'
PASSWORD = '*********'
DATABASE_NAME = 'test'
HOST = 'localhost'
PORT = 3306

async def go(loop):
    try:
        pool = await aiomysql.create_pool(user=DATABASE_USER,
                                         password=PASSWORD,
                                         db=DATABASE_NAME,
                                         host=HOST,
                                         charset='utf8',
                                         autocommit=True,
                                         loop=loop)

        async with pool.acquire() as conn:
            async with conn.cursor() as cur:
                await cur.execute("SELECT * FROM test")
                print(cur.description)
                rs = await cur.fetchall()
                
    finally:
        pool.close()  #数据库连接和cursor同时关闭
        await pool.wait_closed()
        print(cur.closed)
        return rs

In [3]:
rs = loop.run_until_complete(go(loop))

(('id', 3, None, 11, 11, 0, False), ('name', 253, None, 50, 50, 0, False), ('password', 253, None, 50, 50, 0, False))
True


In [4]:
rs

((1, 'tom', '123456'), (2, 'marry', '145867'), (3, 'jim', 'awe123'))

In [ ]:
"""全局连接池,连接池中直接获取数据库连接
所有数据存取完毕时关闭数据库连接
"""
import asyncio
import aiomysql

DATABASE_USER = 'root'
PASSWORD = '*********.'
DATABASE_NAME = 'test'
HOST = 'localhost'
PORT = 3306


class AsyncDatabase:

    def __init__(self,loop=None):
        
        """
        Database operations：mysql
        """
        self.loop = loop or asyncio.get_event_loop()
        # asyncpg
    async def create_pool(self):
        self.conn_pool = await aiomysql.create_pool(
                                                 user=DATABASE_USER,
                                                 password=PASSWORD,
                                                 db=DATABASE_NAME,
                                                 host=HOST,
                                                 charset='utf8',
                                                 autocommit=True,
                                                 loop=self.loop)
    # 最后关闭    
    async def close(self):
        """close db connect
        """
        self.conn_pool.close()
        await self.conn_pool.wait_closed()

    async def select_data(self, sql, size=None):
        await self.create_pool()
        
        async with self.conn_pool.acquire() as conn:
            async with conn.cursor() as cur:
            # cur = await conn.cursor(aiomysql.DictCursor)
                await cur.execute(sql)
                if size:
                    rs = await cur.fetchmany(size)
                else:
                    rs = await cur.fetchall()
                
                return rs